# Download data from apifootball

In [124]:
import requests
import json
from getpass import getpass

In [125]:
api_key = getpass("Insert api key: ")

Insert api key:  ········


In [126]:
base_url = "https://apiv3.apifootball.com/?action=get_events&from={}&to={}&league_id=152&APIkey={}"

In [127]:
request1 = requests.get(base_url.format("2022-08-01", "2022-11-11", api_key))

In [112]:
request2 = requests.get(base_url.format("2022-11-12", "2023-05-29", api_key))

In [113]:
def save_json(request, output_file):
    if request.status_code == 200:
        try:
            json_data = request.json()
            with open(output_file, "w") as f:
                json.dump(json_data, f, indent=4)
            print(f"JSON data saved to {output_file}")
        except json.JSONDecodeError:
            print("Error: The response content is not valid JSON.")
    else:
        print(f"Error: Request failed with status code {response.status_code}")

In [114]:
save_json(request1, "request1.json")
save_json(request2, "request2.json")

JSON data saved to request1.json
JSON data saved to request2.json


# Database design

For this task, it is sufficient to work with two tables:

1. events: raw events data from apifootball (fact table)
2. teams: columns team_id and team_name (dimension)

In [115]:
from pyspark.sql import SparkSession

In [116]:
spark = SparkSession.builder.getOrCreate()

In [117]:
events = spark.read.json("request*.json", multiLine=True)

In [118]:
events.createOrReplaceTempView("events")

In [119]:
spark.sql(
"""
with all_teams as (
    (
        select match_hometeam_id as team_id,
            match_hometeam_name as team_name
        from events
    )
    union all
    (
        select match_awayteam_id as team_id,
            match_awayteam_name as team_name
        from events
    )
)

select team_id,
    min(team_name) as team_name
from all_teams
group by team_id
"""
).createOrReplaceTempView("teams")

# SQL Queries

## Final league table (query_a)

In [137]:
query_a = spark.sql(
"""
with matches_finished as (
    select 
        match_hometeam_id,
        match_awayteam_id,
        cast(match_hometeam_score as int) as match_hometeam_score,
        cast(match_awayteam_score as int) as match_awayteam_score
    from events
    where match_status = 'Finished'
),
matches_points as (
    select match_hometeam_id,
        match_awayteam_id,
        match_hometeam_score,
        match_awayteam_score,
        case
            when match_hometeam_score > match_awayteam_score then 3
            when match_hometeam_score = match_awayteam_score then 1
            else 0
        end as match_hometeam_points,
        case
            when match_hometeam_score < match_awayteam_score then 3
            when match_hometeam_score = match_awayteam_score then 1
            else 0
        end as match_awayteam_points
    from matches_finished
),
team_overall_results as (
    (
        select match_hometeam_id as team_id,
            match_hometeam_points as points,
            match_hometeam_score as goals_scored,
            match_awayteam_score as goals_conceded
        from matches_points
    )
    union all
    (
        select match_awayteam_id as team_id,
            match_awayteam_points as points,
            match_awayteam_score as goals_scored,
            match_hometeam_score as goals_conceded
        from matches_points
    ) 
),
league_table as (
    select team_id,
        count(team_id) as matches_played,
        sum(case when goals_scored > goals_conceded then 1 else 0 end) as won,
        sum(case when goals_scored = goals_conceded then 1 else 0 end) as draw,
        sum(case when goals_scored < goals_conceded then 1 else 0 end) as lost,
        sum(goals_scored) as goals_scored,
        sum(goals_conceded) as goals_conceded,
        sum(points) as points,
        sum(goals_scored)- sum(goals_conceded) as goal_difference
    from team_overall_results
    group by team_id
)

select row_number() over (order by points desc, goal_difference desc, goals_scored desc, goals_conceded, won) as position,
    t.team_name as team_name,
    matches_played,
    won,
    draw,
    lost,
    goals_scored,
    goals_conceded,
    points
from league_table l
left join teams t on l.team_id = t.team_id 
"""
)

query_a.show()

query_a.toPandas().to_csv("query_a.csv", index=False)

23/09/10 19:49:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 1

+--------+--------------------+--------------+---+----+----+------------+--------------+------+
|position|           team_name|matches_played|won|draw|lost|goals_scored|goals_conceded|points|
+--------+--------------------+--------------+---+----+----+------------+--------------+------+
|       1|     Manchester City|            38| 28|   5|   5|          94|            33|    89|
|       2|             Arsenal|            38| 26|   6|   6|          88|            43|    84|
|       3|   Manchester United|            38| 23|   6|   9|          58|            43|    75|
|       4|           Newcastle|            38| 19|  14|   5|          68|            33|    71|
|       5|           Liverpool|            38| 19|  10|   9|          75|            47|    67|
|       6|Brighton & Hove A...|            38| 18|   8|  12|          72|            53|    62|
|       7|         Aston Villa|            38| 18|   7|  13|          51|            46|    61|
|       8|           Tottenham|         

23/09/10 19:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 19:49:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/10 1

## Display all teams sorted by Away goals scored (query_b)

In [138]:
query_b = spark.sql(
"""
with id_goals as (
    select match_awayteam_id as team_id,
        sum(cast(match_awayteam_score as int)) as goals
    from events e
    group by team_id
)
select team_name,
    goals
from id_goals i
left join teams t on i.team_id = t.team_id
order by goals desc, team_name
"""
)

query_b.show()

query_b.toPandas().to_csv("query_b.csv", index=False)

+--------------------+-----+
|           team_name|goals|
+--------------------+-----+
|             Arsenal|   35|
|Brighton & Hove A...|   35|
|     Manchester City|   34|
|           Tottenham|   33|
|           Newcastle|   32|
|           Liverpool|   29|
|      Leicester City|   28|
|              Fulham|   24|
|           Brentford|   23|
|        Leeds United|   22|
|   Manchester United|   22|
|      Crystal Palace|   19|
|         Aston Villa|   18|
|             Chelsea|   18|
|             Everton|   18|
|     AFC Bournemouth|   17|
|         Southampton|   17|
|            West Ham|   16|
|Wolverhampton Wan...|   12|
|   Nottingham Forest|   11|
+--------------------+-----+



## Top 5 Referees with Most Cards (query_c)

In [139]:
query_c = spark.sql(
"""
select match_referee as referee_name,
    sum(size(cards)) as cards
from events
group by match_referee
order by cards desc, referee_name
limit 5
"""
)

query_c.show()

query_c.toPandas().to_csv("query_c.csv", index=False)

+------------+-----+
|referee_name|cards|
+------------+-----+
|   A. Taylor|  124|
|  P. Tierney|  117|
|    R. Jones|  101|
|   S. Hooper|  100|
|   A. Madley|   93|
+------------+-----+



## Top 3 Goal Scorers by Match Round 14 (query_d)

In [140]:
query_d = spark.sql(
"""
with exploded_goals as (
    select match_hometeam_id,
        match_awayteam_id,
        explode(goalscorer) as goal
    from events
    where cast(match_round as int) <= 14
),
scorer as (
    (
        select match_hometeam_id as team_id,
            goal.home_scorer as scorer
        from exploded_goals
        where goal.home_scorer_id <> ''
    )
    union all
    (
        select match_awayteam_id as team_id,
            goal.away_scorer as scorer
        from exploded_goals
        where goal.away_scorer_id <> ''
    )
)
select scorer as player_name,
    team_name,
    count(scorer) as goals
from scorer s
left join teams t on s.team_id = t.team_id
group by team_name, scorer
order by goals desc, player_name
limit 3
"""
)

query_d.show()

query_d.toPandas().to_csv("query_d.csv", index=False)

+-----------+---------------+-----+
|player_name|      team_name|goals|
+-----------+---------------+-----+
| E. Haaland|Manchester City|   19|
|    H. Kane|      Tottenham|   10|
|A. Mitrovic|         Fulham|    9|
+-----------+---------------+-----+

